In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
df = pd.read_csv('C:\\Users\\M\\Machine_L\\data\\titanic_train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#1.전처리
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True) #object타입 드롭
df.Age.fillna(df.groupby('Pclass').Age.transform('mean'), inplace = True) #Age컬럼 평균으로 채우기
#df.Embarked = df.Embarked.dropna() #Embarked컬럼 nan값 삭제

In [6]:
#2. X, y 설정

# onehot인코딩
df_ohe = pd.get_dummies(df, columns = ['Sex', 'Embarked'], dtype = int)

X = df_ohe.drop('Survived', axis = 1)
y = df_ohe.Survived

In [7]:
#3. train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2024)

In [8]:
#4. 모델학습
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier

tree = DecisionTreeClassifier()
log = LogisticRegression(solver='lbfgs', max_iter=1000)
knn = KNeighborsClassifier()
svc = SVC(probability=True)

vot_hard = VotingClassifier(estimators = [('log', log), ('tree', tree), ('knn', knn), ('svc', svc)], voting = 'hard')
vot_soft = VotingClassifier(estimators = [('log', log), ('tree', tree), ('knn', knn), ('svc', svc)], voting = 'soft')

models = [tree, log, knn, svc, vot_hard, vot_soft]

In [9]:
from sklearn.metrics import accuracy_score

for m in models:
  m.fit(X_train, y_train)
  pred = m.predict(X_test)
  print(m.__class__.__name__, accuracy_score(y_test, pred))

DecisionTreeClassifier 0.7541899441340782
LogisticRegression 0.8156424581005587
KNeighborsClassifier 0.7094972067039106
SVC 0.6815642458100558
VotingClassifier 0.7932960893854749
VotingClassifier 0.8100558659217877


In [10]:
#bagging

In [11]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier()
bag.fit(X_train, y_train)
pred = bag.predict(X_test)
print(bag.__class__.__name__, accuracy_score(y_test, pred))

BaggingClassifier 0.7821229050279329


In [12]:
#randomforest

In [13]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(rfc.__class__.__name__, accuracy_score(y_test, pred))

RandomForestClassifier 0.8044692737430168


In [14]:
#AdaBoost

In [15]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
pred = ada.predict(X_test)
print(ada.__class__.__name__, accuracy_score(y_test, pred))

AdaBoostClassifier 0.8268156424581006


In [16]:
# GradientBoosting

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

grad = GradientBoostingClassifier()
grad.fit(X_train, y_train)
pred = grad.predict(X_test)
print(grad.__class__.__name__, accuracy_score(y_test, pred))

GradientBoostingClassifier 0.8156424581005587


In [18]:
# XGBClassifier
# LGBMClassifier(XGB 보완버전)
# CatBoost(category data를 더 잘 학습)

In [19]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [20]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [21]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [22]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
pred = grad.predict(X_test)
print(xgb.__class__.__name__, accuracy_score(y_test, pred))

XGBClassifier 0.8156424581005587


In [23]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
pred = grad.predict(X_test)
print(lgbm.__class__.__name__, accuracy_score(y_test, pred))

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000152 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 207
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [24]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier()
cat.fit(X_train, y_train)
pred = grad.predict(X_test)
print(cat.__class__.__name__, accuracy_score(y_test, pred))

Learning rate set to 0.008911
0:	learn: 0.6874913	total: 137ms	remaining: 2m 17s
1:	learn: 0.6819789	total: 140ms	remaining: 1m 10s
2:	learn: 0.6772158	total: 142ms	remaining: 47.1s
3:	learn: 0.6717360	total: 144ms	remaining: 35.7s
4:	learn: 0.6663083	total: 145ms	remaining: 28.9s
5:	learn: 0.6609520	total: 147ms	remaining: 24.4s
6:	learn: 0.6553094	total: 149ms	remaining: 21.2s
7:	learn: 0.6499345	total: 151ms	remaining: 18.7s
8:	learn: 0.6453829	total: 153ms	remaining: 16.8s
9:	learn: 0.6404608	total: 155ms	remaining: 15.3s
10:	learn: 0.6353267	total: 156ms	remaining: 14.1s
11:	learn: 0.6299476	total: 158ms	remaining: 13s
12:	learn: 0.6248602	total: 160ms	remaining: 12.2s
13:	learn: 0.6203544	total: 162ms	remaining: 11.4s
14:	learn: 0.6165726	total: 164ms	remaining: 10.8s
15:	learn: 0.6119029	total: 166ms	remaining: 10.2s
16:	learn: 0.6071254	total: 167ms	remaining: 9.68s
17:	learn: 0.6040745	total: 169ms	remaining: 9.21s
18:	learn: 0.5994968	total: 173ms	remaining: 8.91s
19:	learn: 